## General Processing

Workflow:
1. General filter
2. Outliers detection
3. Train and test set splitting
4. Missing values imputation
5. Add Score values
6. Output the train and test data




### Load master dataset

In [1]:
import pandas as pd
import os
from helpers import *
from sklearn.impute import SimpleImputer
from dataset_path import output_path,mimic_iv_path
path = "/home/liwens/healthcare/data"
output_path = "/home/liwens/healthcare/data"
df_master = pd.read_csv(os.path.join(path, 'master_dataset_lab.csv'))

In [2]:
pd.set_option('display.max_columns', 200) 
df_master.head()

,index,subject_id,hadm_id,stay_id,intime,outtime,gender,arrival_transport,disposition,anchor_age,anchor_year,dod,admittime,dischtime,deathtime,ethnicity,edregtime,edouttime,insurance,in_year,age,outcome_inhospital_mortality,ed_los,intime_icu,time_to_icu_transfer,outcome_icu_transfer_12h,outcome_hospitalization,outcome_critical,n_ed_30d,n_ed_90d,n_ed_365d,next_ed_visit_time,next_ed_visit_time_diff,outcome_ed_revisit_3d,n_hosp_30d,n_hosp_90d,n_hosp_365d,n_icu_30d,n_icu_90d,n_icu_365d,ed_los_hours,time_to_icu_transfer_hours,next_ed_visit_time_diff_days,triage_temperature,triage_heartrate,triage_resprate,triage_o2sat,triage_sbp,triage_dbp,triage_pain,triage_acuity,chiefcomplaint,chiefcom_chest_pain,chiefcom_abdominal_pain,chiefcom_headache,chiefcom_shortness_of_breath,chiefcom_back_pain,chiefcom_cough,chiefcom_nausea_vomiting,chiefcom_fever_chills,chiefcom_syncope,chiefcom_dizziness,cci_MI,cci_CHF,cci_PVD,cci_Stroke,cci_Dementia,cci_Pulmonary,cci_Rheumatic,cci_PUD,cci_Liver1,cci_DM1,cci_DM2,cci_Paralysis,cci_Renal,cci_Cancer1,cci_Liver2,cci_Cancer2,cci_HIV,eci_CHF,eci_Arrhythmia,eci_Valvular,eci_PHTN,eci_PVD,eci_HTN1,eci_HTN2,eci_Paralysis,eci_NeuroOther,eci_Pulmonary,eci_DM1,eci_DM2,eci_Hypothyroid,eci_Renal,eci_Liver,eci_PUD,eci_HIV,eci_Lymphoma,eci_Tumor2,eci_Tumor1,eci_Rheumatic,eci_Coagulopathy,eci_Obesity,eci_WeightLoss,eci_FluidsLytes,eci_BloodLoss,eci_Anemia,eci_Alcohol,eci_Drugs,eci_Psychoses,eci_Depression,ed_temperature_last,ed_heartrate_last,ed_resprate_last,ed_o2sat_last,ed_sbp_last,ed_dbp_last,ed_pain_last,n_med,n_medrecon,anion gap,bicarbonate,chloride,creatinine,glucose,potassium,sodium,urea nitrogen,eosinophils,hematocrit,hemoglobin,lymphocytes,mch,mchc,mcv,monocytes,neutrophils,platelet count,rdw,red blood cells,white blood cells,absolute eosinophil count,absolute monocyte count,absolute neutrophil count
0,5,10000084,23052089.0,35203156,2160-11-20 20:36:00,2160-11-21 03:20:00,M,WALK IN,ADMITTED,72,2160,2161-02-13,2160-11-21 01:56:00,2160-11-25 14:52:00,NaN,WHITE,2160-11-20 20:36:00,2160-11-21 03:20:00,Medicare,2160,72,False,0 days 06:44:00,NaN,NaN,False,True,False,0,0,0,2160-12-27 18:32:00,36 days 15:12:00,False,0,0,0,0,0,0,6.73,NaN,36.63,97.5,78.0,16.0,100.0,114.0,71.0,0.0,2.0,"Confusion, Hallucinations",False,False,False,False,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,98.0,68.0,16.0,96.0,103.0,74.0,0.0,1.0,5.0,15.0,19.0,98.0,0.8,100.0,5.8,132.0,17.0,2.0,38.2,12.8,33.3,31.8,33.5,95.0,15.9,48.1,263.0,13.0,4.03,8.6,0.17,1.36,4.13
1,6,10000084,29888819.0,36954971,2160-12-27 18:32:00,2160-12-28 16:07:00,M,AMBULANCE,HOME,72,2160,2161-02-13,2160-12-28 05:11:00,2160-12-28 16:07:00,NaN,WHITE,2160-12-27 18:32:00,2160-12-28 16:07:00,Medicare,2160,72,False,0 days 21:35:00,NaN,NaN,False,True,False,0,1,1,NaN,NaN,False,0,1,1,0,0,0,21.58,NaN,NaN,98.7,80.0,16.0,95.0,111.0,72.0,0.0,2.0,"Altered mental status, B Pedal edema",False,False,False,False,False,False,False,False,False,False,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,98.7,80.0,16.0,100.0,119.0,84.0,0.0,1.0,6.0,13.0,24.0,104.0,0.8,108.0,4.1,141.0,18.0,0.5,39.6,13.0,20.0,31.8,32.8,97.0,11.2,67.6,257.0,13.1,4.09,8.7,0.04,0.97,5.87
2,34,10000980,20897796.0,31236252,2193-08-14 21:25:00,2193-08-15 02:22:00,F,WALK IN,ADMITTED,73,2186,2193-08-26,2193-08-15 01:01:00,2193-08-17 15:07:00,NaN,BLACK/AFRICAN AMERICAN,2193-08-14 21:25:00,2193-08-15 02:22:00,Other,2193,80,False,0 days 04:57:00,NaN,NaN,False,True,False,0,0,0,NaN,NaN,False,0,0,0,0,0,0,4.95,NaN,NaN,97.2,NaN,22.0,68.0,100.0,NaN,0.0,1.0,Dyspnea,False,False,False,False,False,False,False,False,False,False,1,1,1,0,0,0,0,1,0,1,1,0,1,0,0,0,0,1,0,1,0,1,1,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,98.2,77.0,18.0,100.0,141.0,76.0,0.0,13.0,9.0,17.0,20.0,105.0,2.4,189.0,4.8,142.0,38.0,1.8,27.8,9.0,17.6,31.3,32.4,97.0,4.4,75.2,176.0,15.1,2.88,6.0,0.11,0.26,4.49
3,37,10000980,25911675.0,39

### 1. General filter - Age, triage_acuity

In [3]:
print('Before filtering for "age" >= 18 : master dataset size = ', len(df_master))
df_master = df_master[df_master['age'] >= 18]
print('After filtering for "age" >= 18 : master dataset size = ', len(df_master))

Before filtering for "age" >= 18 : master dataset size =  21799
After filtering for "age" >= 18 : master dataset size =  21799


In [ ]:
# na_master = df_master[df_master['triage_acuity'].isnull()]
# len(na_master)

In [4]:
print('Before filtering for non-null "triage_acuity" >= 18 : master dataset size = ', len(df_master))
df_master = df_master[df_master['triage_acuity'].notnull()]
print('After filtering for non-null "triage_acuity" >= 18 : master dataset size = ', len(df_master))

Before filtering for non-null "triage_acuity" >= 18 : master dataset size =  21799
After filtering for non-null "triage_acuity" >= 18 : master dataset size =  21746


### 2. Outlier Detection 

In [5]:
# from mimic-extract
vitals_valid_range = {
    'temperature': {'outlier_low': 14.2, 'valid_low': 26, 'valid_high': 45, 'outlier_high':47},
    'heartrate': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 350, 'outlier_high':390},
    'resprate': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 300, 'outlier_high':330},
    'o2sat': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 100, 'outlier_high':150},
    'sbp': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 375, 'outlier_high':375},
    'dbp': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 375, 'outlier_high':375},
    'pain': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 10, 'outlier_high':10},
    'acuity': {'outlier_low': 1, 'valid_low': 1, 'valid_high': 5, 'outlier_high':5},
}

In [6]:
df_master = convert_temp_to_celcius(df_master)

In [7]:

display_outliers_count(df_master, vitals_valid_range)

,variable,< outlier_low,"[outlier_low, valid_low)","[valid_low, valid_high]","(valid_high, outlier_high]",> outlier_high
0,triage_temperature,29,5,20920,0,0
1,triage_heartrate,0,0,21217,0,1
2,triage_resprate,0,0,21115,0,0
3,triage_o2sat,0,0,21040,0,2
4,triage_sbp,0,0,21150,0,0
5,triage_dbp,0,0,21086,0,23
6,triage_pain,0,0,20258,0,63
7,triage_acuity,0,0,21746,0,0
8,ed_temperature_last,55,3,21604,0,1
9,ed_heartrate_last,0,0,21726,0,0


In [8]:

df_master = remove_outliers(df_master, vitals_valid_range)


### 3. Dataset Split (train:0.8, test: 0.2, use seed to fix)

In [9]:
df_train=df_master.sample(frac=0.8,random_state=10) #random state is a seed value
df_test=df_master.drop(df_train.index)

In [10]:
print('Training dataset size = ', len(df_train))
print('Testing dataset size = ', len(df_test))

Training dataset size =  17397
Testing dataset size =  4349


In [11]:
df_train.head()

,index,subject_id,hadm_id,stay_id,intime,outtime,gender,arrival_transport,disposition,anchor_age,anchor_year,dod,admittime,dischtime,deathtime,ethnicity,edregtime,edouttime,insurance,in_year,age,outcome_inhospital_mortality,ed_los,intime_icu,time_to_icu_transfer,outcome_icu_transfer_12h,outcome_hospitalization,outcome_critical,n_ed_30d,n_ed_90d,n_ed_365d,next_ed_visit_time,next_ed_visit_time_diff,outcome_ed_revisit_3d,n_hosp_30d,n_hosp_90d,n_hosp_365d,n_icu_30d,n_icu_90d,n_icu_365d,ed_los_hours,time_to_icu_transfer_hours,next_ed_visit_time_diff_days,triage_temperature,triage_heartrate,triage_resprate,triage_o2sat,triage_sbp,triage_dbp,triage_pain,triage_acuity,chiefcomplaint,chiefcom_chest_pain,chiefcom_abdominal_pain,chiefcom_headache,chiefcom_shortness_of_breath,chiefcom_back_pain,chiefcom_cough,chiefcom_nausea_vomiting,chiefcom_fever_chills,chiefcom_syncope,chiefcom_dizziness,cci_MI,cci_CHF,cci_PVD,cci_Stroke,cci_Dementia,cci_Pulmonary,cci_Rheumatic,cci_PUD,cci_Liver1,cci_DM1,cci_DM2,cci_Paralysis,cci_Renal,cci_Cancer1,cci_Liver2,cci_Cancer2,cci_HIV,eci_CHF,eci_Arrhythmia,eci_Valvular,eci_PHTN,eci_PVD,eci_HTN1,eci_HTN2,eci_Paralysis,eci_NeuroOther,eci_Pulmonary,eci_DM1,eci_DM2,eci_Hypothyroid,eci_Renal,eci_Liver,eci_PUD,eci_HIV,eci_Lymphoma,eci_Tumor2,eci_Tumor1,eci_Rheumatic,eci_Coagulopathy,eci_Obesity,eci_WeightLoss,eci_FluidsLytes,eci_BloodLoss,eci_Anemia,eci_Alcohol,eci_Drugs,eci_Psychoses,eci_Depression,ed_temperature_last,ed_heartrate_last,ed_resprate_last,ed_o2sat_last,ed_sbp_last,ed_dbp_last,ed_pain_last,n_med,n_medrecon,anion gap,bicarbonate,chloride,creatinine,glucose,potassium,sodium,urea nitrogen,eosinophils,hematocrit,hemoglobin,lymphocytes,mch,mchc,mcv,monocytes,neutrophils,platelet count,rdw,red blood cells,white blood cells,absolute eosinophil count,absolute monocyte count,absolute neutrophil count
1092,21573,10520918,26450062.0,31603008,2131-02-17 20:02:00,2131-02-18 02:44:00,F,WALK IN,ADMITTED,24,2131,NaN,2131-02-18 00:55:00,2131-02-22 18:00:00,NaN,OTHER,2131-02-17 20:02:00,2131-02-18 02:44:00,Other,2131,24,False,0 days 06:42:00,2131-02-18 02:44:00,0 days 00:00:00,True,True,True,0,0,0,2133-05-26 22:23:00,828 days 19:39:00,False,0,0,0,0,0,0,6.70,0.00,828.82,36.666667,94.0,22.0,95.0,139.0,77.0,8.0,2.0,"Dyspnea, Chest pain",True,False,False,False,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36.666667,99.0,18.0,100.0,127.0,81.0,0.0,5.0,1.0,34.0,4.0,97.0,0.9,607.0,5.3,130.0,8.0,1.0,41.8,13.6,34.0,30.7,32.5,94.0,0.0,60.0,210.0,13.4,4.43,7.5,0.08,0.00,4.80
16272,318027,17477169,23220329.0,30176036,2135-04-08 20:33:00,2135-04-10 15:21:00,F,WALK IN,HOME,34,2129,NaN,2135-04-09 00:37:00,2135-04-10 15:21:00,NaN,WHITE,2135-04-08 20:33:00,2135-04-10 15:21:00,Medicaid,2135,40,False,1 days 18:48:00,NaN,NaN,False,True,False,0,0,0,2135-10-05 13:23:00,177 days 22:02:00,False,0,0,1,0,0,0,42.80,NaN,177.92,36.166667,91.0,14.0,100.0,141.0,96.0,0.0,3.0,Depression,False,False,False,False,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,36.888889,82.0,16.0,100.0,110.0,68.0,0.0,5.0,6.0,14.0,24.0,102.0,0.7,98.0,3.9,140.0,13.0,0.0,41.0,13.2,27.8,28.2,32.2,88.0,6.2,65.6,284.0,12.4,4.68,8.8,0.00,0.55,5.77
19432,378261,18893057,27798766.0,37800025,2153-08-13 22:33:00,2153-08-14 02:48:37,M,AMBULANCE,ADMITTED,58,2146,NaN,2153-08-14 02:47:00,2153-08-15 19:40:00,NaN,WHITE - OTHER EUROPEAN,2153-08-13 22:33:00,2153-08-14 07:08:00,Medicaid,2153,65,False,0 days 04:15:37,NaN,NaN,False,True,False,0,0,0,2155-05-01 09:49:00,625 days 07:00:23,False,0,0,0,0,0,0,4.26,NaN,625.29,36.722222,82.0,20.0,96.0,136.0,64.0,0.0,2.0,Melena,False,False,False,False,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36.611111,76.0,16.0,95.0,114.0,68.0,0.0,3.0,13.0,18.0,25.0,98.0,0.9,84.0,3.6,137.0,22.0,3.1,37.3,13.1,22.

### 4. Missing Value imputation 

In [12]:
df_missing_stats = df_train.isnull().sum().to_frame().T
df_missing_stats.loc[1] = df_missing_stats.loc[0] / len(df_train)
df_missing_stats.index = ['no. of missing values', 'percentage of missing values']
df_missing_stats

,index,subject_id,hadm_id,stay_id,intime,outtime,gender,arrival_transport,disposition,anchor_age,anchor_year,dod,admittime,dischtime,deathtime,ethnicity,edregtime,edouttime,insurance,in_year,age,outcome_inhospital_mortality,ed_los,intime_icu,time_to_icu_transfer,outcome_icu_transfer_12h,outcome_hospitalization,outcome_critical,n_ed_30d,n_ed_90d,n_ed_365d,next_ed_visit_time,next_ed_visit_time_diff,outcome_ed_revisit_3d,n_hosp_30d,n_hosp_90d,n_hosp_365d,n_icu_30d,n_icu_90d,n_icu_365d,ed_los_hours,time_to_icu_transfer_hours,next_ed_visit_time_diff_days,triage_temperature,triage_heartrate,triage_resprate,triage_o2sat,triage_sbp,triage_dbp,triage_pain,triage_acuity,chiefcomplaint,chiefcom_chest_pain,chiefcom_abdominal_pain,chiefcom_headache,chiefcom_shortness_of_breath,chiefcom_back_pain,chiefcom_cough,chiefcom_nausea_vomiting,chiefcom_fever_chills,chiefcom_syncope,chiefcom_dizziness,cci_MI,cci_CHF,cci_PVD,cci_Stroke,cci_Dementia,cci_Pulmonary,cci_Rheumatic,cci_PUD,cci_Liver1,cci_DM1,cci_DM2,cci_Paralysis,cci_Renal,cci_Cancer1,cci_Liver2,cci_Cancer2,cci_HIV,eci_CHF,eci_Arrhythmia,eci_Valvular,eci_PHTN,eci_PVD,eci_HTN1,eci_HTN2,eci_Paralysis,eci_NeuroOther,eci_Pulmonary,eci_DM1,eci_DM2,eci_Hypothyroid,eci_Renal,eci_Liver,eci_PUD,eci_HIV,eci_Lymphoma,eci_Tumor2,eci_Tumor1,eci_Rheumatic,eci_Coagulopathy,eci_Obesity,eci_WeightLoss,eci_FluidsLytes,eci_BloodLoss,eci_Anemia,eci_Alcohol,eci_Drugs,eci_Psychoses,eci_Depression,ed_temperature_last,ed_heartrate_last,ed_resprate_last,ed_o2sat_last,ed_sbp_last,ed_dbp_last,ed_pain_last,n_med,n_medrecon,anion gap,bicarbonate,chloride,creatinine,glucose,potassium,sodium,urea nitrogen,eosinophils,hematocrit,hemoglobin,lymphocytes,mch,mchc,mcv,monocytes,neutrophils,platelet count,rdw,red blood cells,white blood cells,absolute eosinophil count,absolute monocyte count,absolute neutrophil count
no. of missing values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13766.000000,0.0,0.0,17130.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15181.000000,15181.000000,0.0,0.0,0.0,0.0,0.0,0.0,8735.000000,8735.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15181.000000,8735.000000,656.000000,428.000000,510.000000,565.000000,478.000000,530.000000,1202.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,114.000000,18.000000,19.000000,20.00000,21.000000,28.000000,376.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
percentage of missing values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.791286,0.0,0.0,0.984653,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.872622,0.872622,0.0,0.0,0.0,0.0,0.0,0.0,0.502098,0.502098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.872622,0.502098,0.037708,0.024602,0.029315,0.032477,0.027476,0.030465,0.069092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006553,0.001035,0.001092,0.00115,0.001207,0.001609,0.021613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
vitals_cols = [col for col in df_master.columns if len(col.split('_')) > 1 and 
                                                   col.split('_')[1] in vitals_valid_range and
                                                   col.split('_')[1] != 'acuity']
vitals_cols

['triage_temperature',
 'triage_heartrate',
 'triage_resprate',
 'triage_o2sat',
 'triage_sbp',
 'triage_dbp',
 'triage_pain',
 'ed_temperature_last',
 'ed_heartrate_last',
 'ed_resprate_last',
 'ed_o2sat_last',
 'ed_sbp_last',
 'ed_dbp_last',
 'ed_pain_last']

In [14]:
imputer = SimpleImputer(strategy='median')
df_train[vitals_cols] = imputer.fit_transform(df_train[vitals_cols])
df_test[vitals_cols] = imputer.transform(df_test[vitals_cols])

### 5. Add Score values

In [ ]:
# test
add_triage_MAP(df_test) # add an extra variable MAP
add_score_CCI(df_test)
add_score_CART(df_test)
add_score_REMS(df_test)
add_score_NEWS(df_test)
add_score_NEWS2(df_test)
add_score_MEWS(df_test)

Variable 'add_triage_MAP' successfully added
Variable 'add_score_CCI' successfully added
Variable 'Score_CART' successfully added
Variable 'Score_REMS' successfully added
Variable 'Score_NEWS' successfully added
Variable 'Score_NEWS2' successfully added
Variable 'Score_MEWS' successfully added


In [ ]:
# train
add_triage_MAP(df_train) # add an extra variable MAP
add_score_CCI(df_train)
add_score_CART(df_train)
add_score_REMS(df_train)
add_score_NEWS(df_train)
add_score_NEWS2(df_train)
add_score_MEWS(df_train)

Variable 'add_triage_MAP' successfully added
Variable 'add_score_CCI' successfully added
Variable 'Score_CART' successfully added
Variable 'Score_REMS' successfully added
Variable 'Score_NEWS' successfully added
Variable 'Score_NEWS2' successfully added
Variable 'Score_MEWS' successfully added


In [ ]:
df_train.head(n=10)

,index,subject_id,hadm_id,stay_id,intime,outtime,gender,arrival_transport,disposition,anchor_age,anchor_year,dod,admittime,dischtime,deathtime,ethnicity,edregtime,edouttime,insurance,in_year,age,outcome_inhospital_mortality,ed_los,intime_icu,time_to_icu_transfer,outcome_icu_transfer_12h,outcome_hospitalization,outcome_critical,n_ed_30d,n_ed_90d,n_ed_365d,next_ed_visit_time,next_ed_visit_time_diff,outcome_ed_revisit_3d,n_hosp_30d,n_hosp_90d,n_hosp_365d,n_icu_30d,n_icu_90d,n_icu_365d,ed_los_hours,time_to_icu_transfer_hours,next_ed_visit_time_diff_days,triage_temperature,triage_heartrate,triage_resprate,triage_o2sat,triage_sbp,triage_dbp,triage_pain,triage_acuity,chiefcomplaint,chiefcom_chest_pain,chiefcom_abdominal_pain,chiefcom_headache,chiefcom_shortness_of_breath,chiefcom_back_pain,chiefcom_cough,chiefcom_nausea_vomiting,chiefcom_fever_chills,chiefcom_syncope,chiefcom_dizziness,cci_MI,cci_CHF,cci_PVD,cci_Stroke,cci_Dementia,cci_Pulmonary,cci_Rheumatic,cci_PUD,cci_Liver1,cci_DM1,cci_DM2,cci_Paralysis,cci_Renal,cci_Cancer1,cci_Liver2,cci_Cancer2,cci_HIV,eci_CHF,eci_Arrhythmia,eci_Valvular,eci_PHTN,eci_PVD,eci_HTN1,eci_HTN2,eci_Paralysis,eci_NeuroOther,eci_Pulmonary,eci_DM1,eci_DM2,eci_Hypothyroid,eci_Renal,eci_Liver,eci_PUD,eci_HIV,eci_Lymphoma,eci_Tumor2,eci_Tumor1,eci_Rheumatic,eci_Coagulopathy,eci_Obesity,eci_WeightLoss,eci_FluidsLytes,eci_BloodLoss,eci_Anemia,eci_Alcohol,eci_Drugs,eci_Psychoses,eci_Depression,ed_temperature_last,ed_heartrate_last,ed_resprate_last,ed_o2sat_last,ed_sbp_last,ed_dbp_last,ed_pain_last,n_med,n_medrecon,triage_MAP,score_CCI,score_CART,score_REMS,score_NEWS,score_NEWS2,score_MEWS
183244,183244,14328106,NaN,32336955,2159-11-08 15:13:00,2159-11-09 01:47:00,F,WALK IN,HOME,75,2159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159,75,False,0 days 10:34:00,NaN,NaN,False,False,False,0,0,0,NaN,NaN,False,0,0,0,0,0,0,10.57,NaN,NaN,38.055556,65.0,18.0,100.0,135.0,59.0,0.0,2.0,"Dizziness, Unsteady gait",False,False,False,False,False,False,False,False,False,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36.444444,53.0,16.0,98.0,130.0,68.0,0.0,5.0,9.0,84.333333,3,9,8,1,1,1
376935,376935,18860711,NaN,35614382,2159-10-24 06:19:00,2159-10-24 14:31:00,F,WALK IN,HOME,28,2159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159,28,False,0 days 08:12:00,NaN,NaN,False,False,False,1,1,1,NaN,NaN,False,0,0,0,0,0,0,8.20,NaN,NaN,36.500000,89.0,18.0,97.0,103.0,46.0,10.0,3.0,"Abd pain, n/v/d",False,True,False,False,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36.722222,83.0,16.0,99.0,112.0,51.0,4.0,1.0,5.0,65.000000,0,4,2,1,1,1
291574,291579,16860825,NaN,37485359,2203-01-07 17:45:00,2203-01-08 07:00:00,F,AMBULANCE,HOME,52,2194,NaN,NaN,NaN,NaN,BLACK/AFRICAN AMERICAN,NaN,NaN,NaN,2203,61,False,0 days 13:15:00,NaN,NaN,False,False,False,0,0,5,2203-01-12 22:26:00,4 days 15:26:00,False,0,0,2,0,0,0,13.25,NaN,4.64,36.888889,96.0,16.0,96.0,160.0,80.0,9.0,3.0,"Abd pain, Nausea, Diarrhea",False,True,False,False,False,False,True,False,False,False,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,36.722222,70.0,18.0,98.0,140.0,94.0,3.0,0.0,14.0,106.666667,7,4,3,1,1,1
210522,210524,14971343,29116455.0,34783406,2207-07-11 18:24:00,2207-07-11 22:37:00,F,WALK IN,ADMITTED,74,2198,NaN,2207-07-11 20:24:00,2207-07-12 19:15:00,NaN,BLACK/AFRICAN AMERICAN,2207-07-11 18:24:00,2207-07-11 22:37:00,Medicare,2207,83,False,0 days 04:13:00,NaN,NaN,False,True,False,0,0,0,2208-08-28 08:10:00,413 days 09:33:00,False,0,0,0,0,0,0,4.22,NaN,413.40,36.944444,104.0,16.0,100.0,172.0,89.0,0.0,2.0,"Anxiety, Palpitations",False,False,False,False,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,36.666667,83.0,14.0,99.0,136.0,77.0,0.0,3.0,5.0,116.666667,5,9,8,1,1,2
69270,69316,11648387,NaN,39707189,2181-01-23 09:25:00,2181-01-23 13:27:0

### 6. Output the train and test data

In [ ]:
df_train.to_csv(os.path.join(path, 'train_lab.csv'), index=False)
df_test.to_csv(os.path.join(path, 'test_lab.csv'), index=False)

## Time series data processing

In [ ]:
#mimic_iv_ed_path = 'your_path/mimic-iv-ed-1.0/ed'
df_vitalsign = read_vitalsign_table(os.path.join(mimic_iv_path, 'ed/vitalsign.csv'))

In [ ]:
df_vitalsign['charttime'] = pd.to_datetime(df_vitalsign['charttime'])
df_vitalsign.sort_values('charttime', inplace=True)
df_vitalsign.drop('ed_rhythm', axis=1, inplace=True)
df_vitalsign.head()

,subject_id,stay_id,charttime,ed_temperature,ed_heartrate,ed_resprate,ed_o2sat,ed_sbp,ed_dbp,ed_pain
503777,13238787,35341790,2110-01-11 01:49:00,98.4,77.0,16.0,100.0,149.0,104.0,8.0
836768,15350437,39042378,2110-01-11 03:45:00,97.1,71.0,16.0,100.0,117.0,79.0,0.0
503778,13238787,35341790,2110-01-11 04:02:00,98.0,78.0,18.0,99.0,138.0,92.0,0.0
836769,15350437,39042378,2110-01-11 05:42:00,98.4,57.0,16.0,100.0,132.0,90.0,0.0
836770,15350437,39042378,2110-01-11 08:10:00,99.1,83.0,18.0,100.0,152.0,97.0,0.0


In [ ]:
grouped = df_vitalsign.groupby('stay_id')

In [ ]:
index = pd.date_range('1/1/2000', periods=9, freq='T')
series = pd.Series(range(9), index=index)
print(series.resample('3T'))

DatetimeIndexResampler [freq=<3 * Minutes>, axis=0, closed=left, label=left, convention=start, origin=start_day]


In [ ]:
resample_freq = '1H' # 1 hour
# resample_freq = '30T' # 30 minutes

df_list = []
counter = 0
N = len(grouped)
for stay_id, stay_df in grouped:
    counter += 1
    stay_df.set_index('charttime', inplace=True)
    stay_df = stay_df.resample('1T', origin='start').interpolate(method='linear')\
                     .resample(resample_freq, origin='start').asfreq().ffill().bfill()
    if counter % 10000 == 0 or counter == N:
        print('%d/%d' % (counter, N), end='\r')
    df_list.append(stay_df)

In [ ]:
df_vitalsign_resampled = pd.concat(df_list)

In [ ]:
df_vitalsign_resampled = convert_temp_to_celcius(df_vitalsign_resampled)

In [ ]:
df_vitalsign_resampled = remove_outliers(df_vitalsign_resampled, vitals_valid_range)

In [ ]:
vitals_cols = [col for col in df_vitalsign_resampled.columns if len(col.split('_')) > 1 and 
                                                                col.split('_')[1] in vitals_valid_range]
vitals_cols

['ed_temperature',
 'ed_heartrate',
 'ed_resprate',
 'ed_o2sat',
 'ed_sbp',
 'ed_dbp',
 'ed_pain']

In [ ]:
imputer = SimpleImputer(strategy='median')
df_vitalsign_resampled[vitals_cols] = imputer.fit_transform(df_vitalsign_resampled[vitals_cols])

In [ ]:
df_vitalsign_resampled.to_csv(os.path.join(output_path, 'ed_vitalsign_'+resample_freq+'_resampled.csv'))